In [5]:
import pymongo
from pymongo import MongoClient
import xml.etree.ElementTree as ET

In [6]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client["db1"]
print(client.list_database_names())
print(db.list_collection_names())

['admin', 'config', 'db1', 'local']
['example', 'Users', 'Tags']


In [7]:
## FOR FILES LESS THAN MEMORY SIZE
def XML_to_Mongo_Small(coll_name, file_name):
    c = 0
    root = ET.parse(f'stackoverflow.com/{file_name}').getroot()
    user_coll = db[coll_name]
    users = []
    for typetag in root.findall("row"):
        c += 1
        value = typetag.get('Id')
        print(value)
        user_data = typetag.attrib
        users.append(user_data)
        if(c >= 1000):
            user_coll.insert_many(users)
            users.clear()
            c = 0
    user_coll.insert_many(users)
    users.clear()
    print(user_coll.count_documents({}))

In [21]:
## FOR FILES BIGGER THAN MEMORY SIZE
def XML_to_Mongo_Large(coll_name, file_name):
    c = 0
    user_coll = db[coll_name]
    
    users = []
    for event, typetag in ET.iterparse(f'stackoverflow.com/{file_name}', events=("start", "end")):
        if event == "start":
            c += 1
            value = typetag.get('Id')
            if typetag.tag != "row":
                continue
            user_data = typetag.attrib
            users.append(user_data)
            if(c >= 1000):
                user_coll.insert_many(users)
                users.clear()
                c = 0
        typetag.clear()
    user_coll.insert_many(users)
    users.clear()
    print(user_coll.count_documents({}))

In [22]:
XML_to_Mongo_Large("Posts", "Posts.xml") ## DO THIS ONE BY ONE FOR ALL THE XML FILES

3380601


In [12]:
print(db["Tags"].count_documents({}))

57464


In [26]:
print(db.list_collection_names())

['Votes', 'Posts', 'Users', 'Tags']


In [29]:
for name in db.list_collection_names():
    print(name, db[name].count_documents({}))

Votes 13516747
Posts 3380601
Users 670395
Tags 57464
